# Prepare Video For RAG
Now that you have a good understanding of how we contextualize long-form video assets, in this part of the lab, we will build a Retrieval Augmented Generation (RAG) assistant that can answer questions about the video and help us find relevant clips. This notebook will guide you through the following steps: 1) Contextualize a list of videos using the same technique as the previous notebook. 2) Store scene-level documents with metadata such as video name, start and end timestamps, etc. 3) Upload these documents to S3 as the knowledge source to create a semantic embeddings layer. 4) Create a Bedrock knowledge base using OpenSearch Serverless and ingest the data. 5) Finally, create a RAG assistant to test the videos.

## Pre-req
You must run the [workshop_setup.ipynb](../lab00-setup/workshop_setup.ipynb) notebook in `lab00-setup` before starting this lab.

In [ ]:
import warnings
warnings.warn("Warning: if you did not run lab00-setup, please go back and run the lab00 notebook")

## Load the parameters

In [ ]:
print("load bucket, region, and role....\n")
# bucket and parameter stored from Initial setup lab00
%store -r bucket
%store -r role
%store -r region
%store -r video_prep_prefix

## check all 5 values are printed and do not fail
print(bucket)
print(role)
print(region)
print(video_prep_prefix)

print("\nload the vector db parameters....\n")
# vector parameters stored from Initial setup lab02
%store -r vector_store_name
%store -r vector_collection_arn
%store -r vector_collection_id
%store -r vector_host
%store -r bedrock_kb_execution_role_arn
## check all 4 values are printed and do not fail
print(vector_store_name)
print(vector_collection_arn)
print(vector_collection_id)
print(vector_host)
print(bedrock_kb_execution_role_arn)

In [ ]:
import boto3
import json
import time
from lib import s3_helper as s3h
from termcolor import colored
from lib import util
from lib import video_helper as vh
from lib import ffmpeg_helper as ffh
from pathlib import Path
from IPython.display import display, Video
from sagemaker.utils import name_from_base
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth

scene_doc_dir = "scene_documents"
prefix = video_prep_prefix
util.mkdir(scene_doc_dir)

boto3_session = boto3.Session()
bedrock_agent_client = boto3_session.client('bedrock-agent', region_name=region)


# opensearch service
credentials = boto3_session.get_credentials()
service = 'aoss'
awsauth = auth = AWSV4SignerAuth(credentials, region, service)

### > Video Files To Process

[Meridian](https://opencontent.netflix.com/) is under the [Creative Commons Attribution 4.0 International Public License](https://creativecommons.org/licenses/by/4.0/legalcode)
[ASC StEM2 “The Mission”](https://dpel.aswf.io/asc-stem2/) is under the [ASWF Digital Assets License v1.1](https://raw.githubusercontent.com/AcademySoftwareFoundation/foundation/main/digital_assets/aswf_digital_assets_license_v1.1.txt)

In [ ]:
videos = [
    "https://dx2y1cac29mt3.cloudfront.net/mp4/netflix/Netflix_Open_Content_Meridian.mp4",
    "https://dpel-assets.aswf.io/asc-stem2/ASC_StEM2_178_2K_24_100nits_Rec709_Stereo.mp4"
]

We created helper functions to go through the end-to-end contextualization process quickly

In [ ]:
for video in videos:
    file_name = Path(video).name
    !curl {video} -o {file_name}
    video_dir = Path(video).stem

    display(Video(file_name, width=640, height=360))
    # upload video to S3
    response = s3h.upload_object(bucket, "contextual_ad", file_name)

    stream_info = ffh.probe_stream(file_name)

    # generate chapter segements
    conversations, transcribe_cost, conversation_cost = vh.generate_chapeter_segements(file_name, 
                                                                                        video_dir,
                                                                                        bucket,
                                                                                        stream_info['video_stream']['duration_ms'])

    # generate scene segements
    shots_in_scenes, frames_in_shots, frame_embeddings, frame_embeddings_cost = vh.group_scene_segements(file_name, 
                                                                                                         video_dir, 
                                                                                                         stream_info)

    # align chapter and scenes
    scenes_in_chapters, shots_in_scenes, frames_in_shots, frame_embeddings = vh.align_chapters_n_scenes(video_dir,
                                                                                                        conversations, 
                                                                                                        shots_in_scenes, 
                                                                                                        frames_in_shots, 
                                                                                                        frame_embeddings)

    # contextualize scene and generate scene docs
    frames_in_chapters = vh.get_chapter_frames(frame_embeddings, scenes_in_chapters)
    
    contextual_cost = vh.generate_contextual_output(file_name, 
                                                   video_dir, 
                                                   scene_doc_dir, 
                                                   scenes_in_chapters,
                                                   frames_in_chapters)

    total_estimated_cost = 0

    for estimated_cost in [transcribe_cost, conversation_cost, frame_embeddings_cost, contextual_cost]:
        total_estimated_cost += estimated_cost['estimated_cost']
    total_estimated_cost = round(total_estimated_cost, 4)
    
    print('\n========================================================================\n')
    print('Total estimated cost:', colored(f"${total_estimated_cost}", 'green'))
    print('\n========================================================================')

### > Upload documents to S3

In [ ]:
!aws s3 sync {scene_doc_dir} s3://{bucket}/{prefix}/{scene_doc_dir}

## Create a vector store - OpenSearch Serverless index

For this lab, we will use *Amazon OpenSerach serverless.*

Amazon OpenSearch Serverless is a serverless option in Amazon OpenSearch Service. As a developer, you can use OpenSearch Serverless to run petabyte-scale workloads without configuring, managing, and scaling OpenSearch clusters. You get the same interactive millisecond response times as OpenSearch Service with the simplicity of a serverless environment. Pay only for what you use by automatically scaling resources to provide the right amount of capacity for your application—without impacting data ingestion.

In [ ]:
aoss_client = boto3_session.client('opensearchserverless')

### Create the schema for vector index

In [ ]:
index_name = name_from_base("video-prep")
body_json = {
   "settings": {
      "index.knn": "true"
   },
   "mappings": {
      "properties": {
         "vector": {
            "type": "knn_vector",
            "dimension": 1024,
            "method": {
                "name": "hnsw",
                "space_type": "innerproduct",
                "engine": "faiss",
                "parameters": {
                  "ef_construction": 256,
                  "m": 48
                }
             }
         },
         "text": {
            "type": "text"
         },
         "text-metadata": {
            "type": "text"         
         }
      }
   }
}
# Build the OpenSearch client
oss_client = OpenSearch(
    hosts=[{'host': vector_host, 'port': 443}],
    http_auth=awsauth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
    timeout=300
)

### Create the index in OSS collection

In [ ]:
# Create index
response = oss_client.indices.create(index=index_name, body=json.dumps(body_json))
print('\nCreating index:')
print(response)

## Create Knowledge Base
Steps:
- initialize Open search serverless configuration which will include collection ARN, index name, vector field, text field and metadata field.
- initialize chunking strategy, based on which KB will split the documents into pieces of size equal to the chunk size mentioned in the `chunkingStrategyConfiguration`.
- initialize the s3 configuration, which will be used to create the data source object later.
- initialize the Titan embeddings model ARN, as this will be used to create the embeddings for each of the text chunks.

In [ ]:
opensearchServerlessConfiguration = {
            "collectionArn": vector_collection_arn,
            "vectorIndexName": index_name,
            "fieldMapping": {
                "vectorField": "vector",
                "textField": "text",
                "metadataField": "text-metadata"
            }
        }

chunkingStrategyConfiguration = {
    "chunkingStrategy": "NONE",
}

s3Configuration = {
    "bucketArn": f"arn:aws:s3:::{bucket}",
    "inclusionPrefixes":[f"{prefix}/{scene_doc_dir}/"] # you can use this if you want to create a KB using data within s3 prefixes.
}

embeddingModelArn = f"arn:aws:bedrock:{region}::foundation-model/amazon.titan-embed-text-v2:0"

kb_name = name_from_base("video-knowledge-base")
description = "Video Search knowledge base."

Provide the above configurations as input to the `create_knowledge_base` method, which will create the Knowledge base.

In [ ]:
# Create a KnowledgeBase
from retrying import retry

@retry(wait_random_min=1000, wait_random_max=2000,stop_max_attempt_number=7)
def create_knowledge_base_func():
    create_kb_response = bedrock_agent_client.create_knowledge_base(
        name = kb_name,
        description = description,
        roleArn = bedrock_kb_execution_role_arn,
        knowledgeBaseConfiguration = {
            "type": "VECTOR",
            "vectorKnowledgeBaseConfiguration": {
                "embeddingModelArn": embeddingModelArn
            }
        },
        storageConfiguration = {
            "type": "OPENSEARCH_SERVERLESS",
            "opensearchServerlessConfiguration":opensearchServerlessConfiguration
        }
    )
    return create_kb_response["knowledgeBase"]

In [ ]:
try:
    kb = create_knowledge_base_func()
except Exception as err:
    print(f"{err=}, {type(err)=}")

Next we need to create a data source, which will be associated with the knowledge base created above. Once the data source is ready, we can then start to ingest the documents.

In [ ]:
# Get KnowledgeBase 
get_kb_response = bedrock_agent_client.get_knowledge_base(knowledgeBaseId = kb['knowledgeBaseId'])

In [ ]:
# Create a DataSource in KnowledgeBase 
create_ds_response = bedrock_agent_client.create_data_source(
    name = kb_name,
    description = description,
    knowledgeBaseId = kb['knowledgeBaseId'],
    dataSourceConfiguration = {
        "type": "S3",
        "s3Configuration":s3Configuration
    },
    vectorIngestionConfiguration = {
        "chunkingConfiguration": chunkingStrategyConfiguration
    }
)
ds = create_ds_response["dataSource"]
# # It can take up to a minute for data access rules to be enforced
time.sleep(20)

### Start ingestion job
Once the KB and data source is created, we can start the ingestion job.
During the ingestion job, KB will fetch the documents in the data source, pre-process it to extract text, chunk it based on the chunking size provided, create embeddings of each chunk and then write it to the vector database, in this case OSS.

In [ ]:
# Start an ingestion job
start_job_response = bedrock_agent_client.start_ingestion_job(knowledgeBaseId = kb['knowledgeBaseId'], dataSourceId = ds["dataSourceId"])

In [ ]:
job = start_job_response["ingestionJob"]
print(job)

In [ ]:
# Get job 
while(job['status']!='COMPLETE' ):
  get_job_response = bedrock_agent_client.get_ingestion_job(
      knowledgeBaseId = kb['knowledgeBaseId'],
        dataSourceId = ds["dataSourceId"],
        ingestionJobId = job["ingestionJobId"]
  )
  job = get_job_response["ingestionJob"]
print(job)
time.sleep(80)

In [ ]:
kb_id = kb["knowledgeBaseId"]
%store kb_id
print(kb_id)

## Test the knowledge base
### Using RetrieveAndGenerate API
Behind the scenes, RetrieveAndGenerate API converts queries into embeddings, searches the knowledge base, and then augments the foundation model prompt with the search results as context information and returns the FM-generated response to the question. For multi-turn conversations, Knowledge Bases manage short-term memory of the conversation to provide more contextual results.

The output of the RetrieveAndGenerate API includes the generated response, source attribution as well as the retrieved text chunks.

In [ ]:
# try out KB using RetrieveAndGenerate API
bedrock_agent_runtime_client = boto3.client("bedrock-agent-runtime", region_name=region)
model_id = "anthropic.claude-3-sonnet-20240229-v1:0" # try with both claude instant as well as claude-v2. for claude v2 - "anthropic.claude-v2"
model_arn = f'arn:aws:bedrock:{region}::foundation-model/{model_id}'

Utility function to help display the video

In [ ]:
from IPython.display import display, HTML

def display_video(video_path, start_time=0, width=640, height=360):
    control_id = name_from_base("id")

    display(HTML(f"""
    <video alt="test" controls id="{control_id}" width="{width}" height="{height}" >
      <source src="{video_path}">
    </video>
    
    <script>
    video = document.getElementById("{control_id}")
    video.currentTime = {start_time};
    </script>
    """))

### > Sample Questions
- from ASC_StEM2_178_2K_24_100nits_Rec709_Stereo, where is the scene where group of people running through a rugged and arid desert
- from ASC_StEM2_178_2K_24_100nits_Rec709_Stereo, find me a car is running away from a hotstil situation.
- from the Meridian video, show me scene where 'Virgin Soil Pictures Production' logo appears
- from the Meridian video, show me scene when the live credits scene appears
- from the Meridian video, show me an office setting where people having serious discussion

In [ ]:
from IPython.display import Markdown, display

query = "from the Meridian video, show me an office setting where people having serious discussion"
response = bedrock_agent_runtime_client.retrieve_and_generate(
    input={
        'text': query
    },
    retrieveAndGenerateConfiguration={
        'type': 'KNOWLEDGE_BASE',
        'knowledgeBaseConfiguration': {
            'knowledgeBaseId': kb_id,
            'modelArn': model_arn
        }
    },
)

generated_text = response['output']['text']

display(Markdown(generated_text))

if "citations" in response:
    references = response["citations"][0]["retrievedReferences"]
    if len(references)>0:
        for ref in references:
            video = ref['metadata']['video']
            start = ref['metadata']['start']
            print(video, start)
            display_video(video, start_time=int(start))